In [1]:
# Load the TensorBoard notebook extension

#Uncomment to use it in colab and monitor to tendorboard
#%load_ext tensorboard

import tensorflow as tf
import datetime

##Data preparation

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import kerastuner as kt
from contextlib import redirect_stdout
import shutil

Define the path to load all locally stored csv with data from yahoo:

In [3]:
path = os.getcwd()+"/exper_files/datasets/" #Define where are the datasets

Create the dataset for each input layer:

In [4]:
def dataset_creation(crypto_list, pth):
    datasets = {} #Store all datasets here
    min_dates = {} #Store all minimum dates here
    max_dates = {} #Store all maximum dates here
    datasets_list = os.listdir(pth) #Find all dataset
    
    crypto_list = [elem+'.csv' for elem in crypto_list]

    for dataset in datasets_list:
        if dataset.endswith(".csv") and (dataset) in crypto_list:

            name = dataset.split(".")
            dataset_name = name[0]

            datasets[dataset_name] = pd.read_csv(pth + dataset) 

            datasets[dataset_name]['Date'] = pd.to_datetime(datasets[dataset_name]['Date'])
            datasets[dataset_name].fillna(method='ffill', inplace=True)

            #Create new columns 'close_off_high' and 'volatility' in order to make predictions more accurate:
            kwards = {'close_off_high': lambda x: 2 * (x['High'] - x['Close']) / (x['High'] - x['Low']) - 1,
              'volatility': lambda x: (x['High'] - x['Low']) / (x['Open'])
              }

            datasets[dataset_name] = datasets[dataset_name].assign(**kwards)
            
            datasets[dataset_name].fillna(-1, inplace=True) #Fill possible NaN close_off_high values with -1
                                                            #Because NaN values caused by (0/0)-1

            first_date = pd.to_datetime(datasets[dataset_name]['Date'][0])
            min_dates[dataset_name] = first_date
            
            last_date = pd.to_datetime(datasets[dataset_name]['Date'].iloc[-1])
            max_dates[dataset_name] = last_date

    max_date = max(min_dates.values(), key=lambda v: v)
    min_date = min(max_dates.values(), key=lambda v: v)

    #Drop all the data which are prior to max_date and later to min_date
    for dataset in datasets:
        datasets[dataset] = datasets[dataset][(datasets[dataset]['Date'] >= max_date) & 
                                              (datasets[dataset]['Date'] <= min_date)]


    #Compute the average and standard deviation of 'Close' value for the last 7-days and 30-days(month): 
    for dataset in datasets:

        temp = datasets[dataset].copy()

        #Drop the first 30 days to be able to compute average and standard deviation of month for the rows of the table
        temp = temp[29:]

        temp['mean_7days_Close'] = datasets[dataset]['Close'].rolling(window=7).mean()

        temp['mean_month_Close'] = datasets[dataset]['Close'].rolling(window=30).mean()

        temp['std_7days_Close'] = datasets[dataset]['Close'].rolling(window=7).std()

        temp['std_month_Close'] = datasets[dataset]['Close'].rolling(window=30).std()

        datasets[dataset] = temp.copy()
        



    #Rename the columns
    for dataset in datasets:

        datasets[dataset] = datasets[dataset].rename(columns={'Close':dataset+'_Close', 'Open':dataset+'_Open', 
                                                              'High':dataset+'_High', 'Low':dataset+'_Low', 
                                                              'Adj Close':dataset+'_Adj_Close', 
                                                              'Volume':dataset+'_Volume', 
                                                              'close_off_high':dataset+'_close_off_high',
                                                              'volatility':dataset+'_volatility',
                                                              'mean_7days_Close':dataset+'_mean_7days_Close',
                                                              'mean_month_Close':dataset+'_mean_month_Close',
                                                              'std_7days_Close':dataset+'_std_7days_Close',
                                                              'std_month_Close':dataset+'_std_month_Close'})
        
    
   
    return datasets

Split data into training, validation and test set:

In [5]:
def split_data(perc_train_set, perc_val_set, dict_currency_data):
        
        dict_tr_set = {}
        dict_val_set = {}
        dict_tst_set = {}
        
        for coin, currency_data in dict_currency_data.items():
            
            #Compute the date to split the dataset into training and validation_test set based on 'perc_train_set'
            splt_date_train = currency_data.iloc[round(currency_data.shape[0] * perc_train_set)]['Date']

            #Split the dataset into trainning and validation_test set
            tr_set, val_tst_set = currency_data[currency_data['Date'] < splt_date_train], \
                             currency_data[currency_data['Date'] >= splt_date_train]

            #Compute the date to split the val_tst_set into validation and test set based on 'perc_val_set'
            splt_date_val = val_tst_set.iloc[round(val_tst_set.shape[0] * perc_val_set)]['Date']

            #Split the val_tst_set into validation and test set        
            val_set, tst_set = val_tst_set[val_tst_set['Date'] < splt_date_val], \
                                val_tst_set[val_tst_set['Date'] >= splt_date_val]
            
            
            
            #Drop 'Date' column and save train, validation and test sets of each coin at the corresponding dictionary 
            tr_set = ((tr_set.drop(['Date'], axis=1)).reset_index()).drop(['index'], axis=1)
            dict_tr_set[coin] = tr_set
            
            val_set = ((val_set.drop(['Date'], axis=1)).reset_index()).drop(['index'], axis=1)
            dict_val_set[coin] = val_set
            
            tst_set = ((tst_set.drop(['Date'], axis=1)).reset_index()).drop(['index'], axis=1)
            dict_tst_set[coin] = tst_set
        
        return dict_tr_set, dict_val_set, dict_tst_set, splt_date_train, splt_date_val

Normalize training, validation and test inputs and outputs with sliding window:

In [6]:
def normalize_in_out(prd_range, wind_len, dict_tr_set, dict_val_set, dict_tst_set, target_feats, coin_targ, crypto_list):
    
    dict_LSTM_tr_in = {}
    dict_LSTM_val_in = {}
    dict_LSTM_test_in = {}
    
    
    for crypto in crypto_list:
        
        tr_set = dict_tr_set[crypto]
        val_set = dict_val_set[crypto]
        tst_set = dict_tst_set[crypto]
        
        all_feats = tr_set.columns #Get all features
        feats = [crypto+"_"+feat for feat in target_feats] #Get the features in the appropriate format 
                                                           #(e.g 'Close' --> 'BTC-USD_Close')
        
        
        #Normalize training inputs
        LSTM_tr_in = []
        for i in range(len(tr_set) - wind_len):
            tmp_set = tr_set[i:(i + wind_len)].copy()

            for col in all_feats:
                if col not in feats:
                    tmp_set = tmp_set.drop([col], axis=1) #Drop the feature that will not be used

            for col in feats:
                tmp_set[:][col] = tmp_set[col] / tmp_set[col].iloc[0] - 1 #Normalize the feature that will be used

            LSTM_tr_in.append(tmp_set)

        #Transform from DataFrame to numpy array
        LSTM_tr_in = [np.array(LSTM_tr_i) for LSTM_tr_i in LSTM_tr_in]
        LSTM_tr_in = np.array(LSTM_tr_in)
        
        dict_LSTM_tr_in[crypto] = LSTM_tr_in


        #Normalize validation inputs
        LSTM_val_in = []
        for i in range(len(val_set) - wind_len):
            tmp_set = val_set[i:(i + wind_len)].copy()

            for col in all_feats:
                if col not in feats:
                    tmp_set = tmp_set.drop([col], axis=1) #Drop the feature that will not be used

            for col in feats:
                tmp_set[:][col] = tmp_set[col] / tmp_set[col].iloc[0] - 1 #Normalize the feature that will be used

            LSTM_val_in.append(tmp_set)

        #Transform from DataFrame to numpy array
        LSTM_val_in = [np.array(LSTM_val_i) for LSTM_val_i in LSTM_val_in]
        LSTM_val_in = np.array(LSTM_val_in)
        
        dict_LSTM_val_in[crypto] = LSTM_val_in

        #Normalize test inputs
        LSTM_test_in = []
        for i in range(len(tst_set) - wind_len):
            tmp_set = tst_set[i:(i + wind_len)].copy() 

            for col in all_feats:
                if col not in feats:

                    tmp_set = tmp_set.drop([col], axis=1) #Drop the feature that will not be used

            for col in feats:
                tmp_set[:][col] = tmp_set[col] / tmp_set[col].iloc[0] - 1 #Normalize the feature that will be used

            LSTM_test_in.append(tmp_set)


        #Transform from DataFrame to numpy array
        LSTM_test_in = [np.array(LSTM_test_i) for LSTM_test_i in LSTM_test_in]
        LSTM_test_in = np.array(LSTM_test_in)
        
        dict_LSTM_test_in[crypto] = LSTM_test_in 
    
    
    tr_set = dict_tr_set[coin_targ].copy()
    val_set = dict_val_set[coin_targ].copy()
    
    #Normalize training outputs
    LSTM_rangd_train_out = []
    for i in range(wind_len, len(tr_set[coin_targ+'_Close']) - prd_range):
        LSTM_rangd_train_out.append((tr_set[coin_targ+'_Close'][i:i+prd_range].values/tr_set[coin_targ+'_Close'][tr_set.index[0]+i-wind_len]) - 1)

    LSTM_rangd_train_out = np.array(LSTM_rangd_train_out)
    
    
    #Normalize validation outputs
    LSTM_rangd_val_out = []
    for i in range(wind_len, len(val_set[coin_targ+'_Close']) - prd_range):
        LSTM_rangd_val_out.append((val_set[coin_targ+'_Close'][i:i+prd_range].values/val_set[coin_targ+'_Close'][val_set.index[0]+i-wind_len]) - 1)
    
    LSTM_rangd_val_out = np.array(LSTM_rangd_val_out)
    
    return LSTM_rangd_train_out, LSTM_rangd_val_out, dict_LSTM_tr_in, dict_LSTM_val_in, dict_LSTM_test_in

Create MAE plot:

In [7]:
def create_plot(rangd_h, path_to_sav, target_cn):
    
    fig, ax1 = plt.subplots(1, 1, figsize=(25, 10))

    ax1.plot(rangd_h.epoch, rangd_h.history['loss'], color='b', label='Loss')
    ax1.plot(rangd_h.epoch, rangd_h.history['val_loss'], color='orange', label='Val_loss')
    ax1.set_xticks(range(0, len(rangd_h.epoch)))
    ax1.set_title('Training Error')
    ax1.set_ylabel('Mean Absolute Error (MAE)', fontsize=12)
    ax1.set_xlabel('#Epoch', fontsize=12)
    #plt.legend()
    plt.savefig(path_to_sav+"/"+target_cn+'_MAE.png')
    plt.close(fig)
    #plt.show()

Define, build and train model

In [8]:
########Model with CNN layers, followed by LSTM layers, followed by Dense layers #####################
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
import tensorflow as tf
from keras.layers import Conv1D
from keras.layers import MaxPool1D
from keras.layers import AveragePooling1D
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.models import Model
from keras import Input


def model_tuning(coin_list, inputs, outputs, output_size, validation_inputs, validation_outputs, epochs, early_stop_patience, 
                 neurons, dropout, prd_range, batch_size, keras_tuner_dir, activ_func="linear", loss="mae", 
                 optimizer="adam"):
    
    def build_model(hp):
        
        separate_coin_layers = {}
        
        neurLSTM1 = hp.Choice('neurons_1_LSTM', values=neurons)
        
        for coin in coin_list:
                       
            #Input layer for each coin with each own data
            separate_coin_layers[coin] = {}
            separate_coin_layers[coin]['input'] = Input(shape=((inputs[coin].shape[1], inputs[coin].shape[2])),
                                                             name="input_"+coin)
            #model = Sequential()

            #filters_conv1 = hp.Choice('filters_Conv1', values=[16, 32, 64])
            #kernel_size_conv1 = hp.Choice('kernel_size_Conv1', values=[2, 3, 5])
            #strides_conv1 = hp.Choice('strides_Conv1', values=[1, 2, 3])
            #model.add(Conv1D(filters=filters_conv1, kernel_size=kernel_size_conv1, strides=strides_conv1, 
            #                 activation='relu',
            #                 input_shape=(inputs.shape[1], inputs.shape[2])))


            #moment_batchNormConv1 = hp.Float('momentum_batchNormConv1', min_value=0.09, max_value=0.99, step=0.1)
            #model.add(BatchNormalization(momentum=moment_batchNormConv1))

            #pool_size_maxPool1 = hp.Choice('pool_size_MaxPool1', values=[2, 3, 4])
            #model.add(MaxPool1D(pool_size=2))
            #model.add(AveragePooling1D(pool_size=2))

            
            separate_coin_layers[coin]['LSTM1'] = LSTM(neurLSTM1)(separate_coin_layers[coin]['input']) #, input_shape=(inputs[coin].shape[1], inputs[coin].shape[2]) #return_sequences=True

            #dropLSTM1 = hp.Choice('dropout_LSTM_1_'+coin, values=dropout)
            #model.add(Dropout(dropLSTM1))

            #neurLSTM2 = hp.Choice('neurons_2_LSTM', values=neurons)
            #model.add(LSTM(neurLSTM2))

            #dropLSTM2 = hp.Choice('dropout_LSTM_2', values=dropout)
            #model.add(Dropout(dropLSTM2))

            #moment_batchNormLSTM2 = hp.Float('momentum_batchNormLSTM2', min_value=0.09, max_value=0.99, step=0.1)
            #model.add(BatchNormalization(momentum=moment_batchNormLSTM2))

                      
        # Merge all available features into a single large vector via concatenation
        conc_layer_output = Concatenate()([separate_coin_layers[coin]['LSTM1'] for coin in coin_list])
        
        neurD1 = hp.Choice('neurons_1_Dense', values=[32, 64, 128, 256])
        dense1_output = Dense(units=neurD1, activation='relu')(conc_layer_output)
        
        #dropDense1 = hp.Choice('dropout_Dense_1', values=dropout)
        #model.add(Dropout(dropDense1))
        
        #moment_batchNormDense1 = hp.Float('momentum_batchNormDense1', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormDense1))
        
        neurD2 = hp.Choice('neurons_2_Dense', values=[32, 64, 128, 256])
        dense2_output = Dense(units=neurD2, activation='relu')(dense1_output)
        
        #dropDense2 = hp.Choice('dropout_Dense_2', values=dropout)
        #model.add(Dropout(dropDense2))
        
        #moment_batchNormDense2 = hp.Float('momentum_batchNormDense2', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormDense2))
        
        final_output = Dense(units=output_size, activation=activ_func)(dense2_output)
        
        overall_model = Model([separate_coin_layers[coin]['input'] for coin in coin_list], final_output)
        
        hp_learning_rate = hp.Float(name='learning_rate', min_value=0.0001, max_value=0.05, step=0.0005)
        
        hp_beta_1 = hp.Float('beta_1', min_value=0.85, max_value=0.95, step=0.01)
        
        hp_beta_2 = hp.Float('beta_2', min_value=0.98, max_value=0.999, step=0.001)
        
        hp_epsilon = hp.Float('epsilon', min_value=1e-07, max_value=1e-08, sampling='LOG')
        
        optimizer = tf.keras.optimizers.Adam(
        learning_rate=hp_learning_rate, beta_1=hp_beta_1, beta_2=hp_beta_2, epsilon=hp_epsilon)
        
        overall_model.compile(loss=loss, optimizer=optimizer)
        return overall_model
    
    
    
    #Create_tuner
    tuner = kt.Hyperband(build_model,
                 objective='val_loss',
                 max_epochs=epochs,
                 factor=3,
                 directory=keras_tuner_dir,
                 project_name='keras_tuner')

    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=early_stop_patience)

    tuner.search([inputs[coin][:-prd_range] for coin in coin_list], outputs, 
                 validation_data=([validation_inputs[coin][:-prd_range] for coin in coin_list], validation_outputs), 
                 epochs=epochs, batch_size=batch_size, callbacks=[stop_early],
                 use_multiprocessing=True,
                 workers=8)

    # Get the optimal hyperparameters
    best_hps=tuner.get_best_hyperparameters(num_trials=3)[0]

    
    return best_hps, tuner

Overall run:

In [9]:
#Get data
crypto_list = [['BTC-USD', 'ETH-USD', 'DOGE-USD', 'USDT-USD','XRP-USD', 'LTC-USD', 'XLM-USD']] #Define the sets of cryptocurrencies to be tested

first_txt_flag = 0


for cryptocurrency_list in crypto_list:
    
    cryptocurrency_list_output = 'Using cryptocurrencies: '+str(cryptocurrency_list)
    print(cryptocurrency_list_output)
    data = dataset_creation(cryptocurrency_list, path)

    #Split data
    percent_train_set = 0.8
    percent_val_set = 0.5

    training_set, validation_set, test_set, split_date_train, split_date_valid = split_data(percent_train_set, 
                                                                                           percent_val_set, data)

    
    features_list = [['Close', 'Open', 'High']]
    """[['Close'], ['Close', 'Volume'], ['Close', 'Open', 'High'],
                     ['Close', 'close_off_high', 'volatility'],
                     ['Close', 'mean_7days_Close', 'mean_month_Close'],
                     ['Close', 'std_7days_Close', 'std_month_Close']]""" #Define the sets of features to be tested 
    
    for featurs in features_list: 
        
        new_txt_flag = 0 
        
        featurs_output = '\tUsing the features: '+str(featurs)
        print('\tUsing the features: '+str(featurs))
        
        
        #Create inputs and outputs for the model training, validation and testing
        pred_range = 5
        window_len = 10
        features = featurs
        coin_target = 'BTC-USD'
        
        

        LSTM_ranged_training_outputs, LSTM_ranged_validation_outputs, LSTM_training_inputs, LSTM_validation_inputs, LSTM_test_inputs = normalize_in_out(
                                                                                                      pred_range, window_len, 
                                                                                                      training_set, 
                                                                                                      validation_set, 
                                                                                                      test_set, 
                                                                                                      features, coin_target,
                                                                                                      cryptocurrency_list)

        
        batch_size_list = [64] #[1, 32, 64]
        neuron_list = [20, 40, 60, 100]
        dropout_list = [0.0, 0.1, 0.2, 0.25, 0.3, 0.4]
        early_stop_patience = 10
        epochs = 100
        
        
        
        
        
        for bat_s in batch_size_list:

            #Build and train model
            epochs = 500
            batch_size = bat_s
            early_stop_patience = 10
            shuffle = True
            verbose = 0
            
            exper_params_output = '\t\tBatch_size: '+str(batch_size)
            print(exper_params_output)
           
            
            #Path to write log files
            cur_path = os.getcwd()+'/'
            log_dir = cur_path + "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/' + 'Batch_size='+str(batch_size)
            if not os.path.exists(log_dir):
                os.makedirs(log_dir)
            
            #Path to save its checkpoints the keras_tuner because it raises an error when it is in the current working
            #directory or in a sub-folder
            keras_tuner_dir = 'C:\keras_tuner'
            if not os.path.exists(keras_tuner_dir):
                os.makedirs(keras_tuner_dir)
            
            best_hps, tuner = model_tuning(cryptocurrency_list, inputs=LSTM_training_inputs, outputs=LSTM_ranged_training_outputs,
                                output_size=pred_range, 
                                validation_inputs = LSTM_validation_inputs, 
                                validation_outputs = LSTM_ranged_validation_outputs,
                                epochs=epochs, early_stop_patience=early_stop_patience, neurons=neuron_list, 
                                dropout=dropout_list, prd_range=pred_range, batch_size=bat_s, 
                                           keras_tuner_dir=keras_tuner_dir)
        
            exper_outputs = "\t\tThe hyperparameter search is complete. The optimal parameter were found to be: \n" + \
                            str(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)
        
            
            
            # Build the model with the optimal hyperparameters and train it
            best_model = tuner.hypermodel.build(best_hps)
            mod_history = best_model.fit([LSTM_training_inputs[coin][:-pred_range] for coin in cryptocurrency_list], 
                                         LSTM_ranged_training_outputs, 
                                         validation_data=([LSTM_validation_inputs[coin][:-pred_range] 
                                                           for coin in cryptocurrency_list], 
                                                          LSTM_ranged_validation_outputs), 
                                         epochs=epochs)
            
            #Get the epoch with the best validation loss
            val_loss_per_epoch = mod_history.history['val_loss']
            best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
            exper_outputs += '\n\n\t\tBest epoch: '+ str(best_epoch) + ' with validation loss: ' + str((min(val_loss_per_epoch)))
            
            #Print results to a .txt file
            if first_txt_flag == 0:
                
                if new_txt_flag == 0:
                    
                    #Create a .txt file and write the results
                    txt_log_dir = "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/'
                    f = open(txt_log_dir+"/"+coin_target+"_results.txt", "w")
                    f.write(cryptocurrency_list_output)
                    f.write("\n"+featurs_output)
                    
                    first_txt_flag += 1
                    new_txt_flag += 1
            else:
                
                if new_txt_flag == 0:
                    
                    f.close() #Close previous txt file
                    
                    #Create a .txt file and write the results
                    txt_log_dir = "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/'
                    f = open(txt_log_dir+"/"+coin_target+"_results.txt", "w")
                    f.write(cryptocurrency_list_output)
                    f.write("\n"+featurs_output)
                    
                    new_txt_flag += 1
                    
                
            
            f.write("\n"+exper_params_output)
            f.write("\n"+exper_outputs)
            
            f.write("\n\nFull keras tuner results: \n")
            with redirect_stdout(f): 
                tuner.results_summary()
            
            
            create_plot(mod_history, log_dir, coin_target)

f.close()


#Write best model summary to a txt file
with open(cur_path + 'logs/fit/modelsummary.txt', 'w') as f_sum:
    with redirect_stdout(f_sum):
        best_model.summary()
        

#Remove folder with is created by keras tuner
shutil.rmtree(keras_tuner_dir)

Trial 725 Complete [00h 00m 22s]
val_loss: 0.03547787666320801

Best val_loss So Far: 0.03500007092952728
Total elapsed time: 03h 28m 07s
INFO:tensorflow:Oracle triggered exit
Epoch 1/500
52/52 [==============================] - 12s 54ms/step - loss: 0.0786 - val_loss: 0.0412
Epoch 2/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0560 - val_loss: 0.0387
Epoch 3/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0509 - val_loss: 0.0381
Epoch 4/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0485 - val_loss: 0.0366
Epoch 5/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0502 - val_loss: 0.0370
Epoch 6/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0466 - val_loss: 0.0382
Epoch 7/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0460 - val_loss: 0.0363
Epoch 8/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0472 - val_loss: 0.0360
Epoch 9/500
52/52 [====

52/52 [==============================] - 1s 16ms/step - loss: 0.0125 - val_loss: 0.0440
Epoch 161/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0131 - val_loss: 0.0440
Epoch 162/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0132 - val_loss: 0.0461
Epoch 163/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0132 - val_loss: 0.0453
Epoch 164/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0135 - val_loss: 0.0443
Epoch 165/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0137 - val_loss: 0.0466
Epoch 166/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0130 - val_loss: 0.0435
Epoch 167/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0130 - val_loss: 0.0459
Epoch 168/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0130 - val_loss: 0.0463
Epoch 169/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0138 - val_loss: 0

52/52 [==============================] - 1s 15ms/step - loss: 0.0090 - val_loss: 0.0445
Epoch 321/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0090 - val_loss: 0.0443
Epoch 322/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0090 - val_loss: 0.0451
Epoch 323/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0090 - val_loss: 0.0438
Epoch 324/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0088 - val_loss: 0.0442
Epoch 325/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0086 - val_loss: 0.0440
Epoch 326/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0087 - val_loss: 0.0455
Epoch 327/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0095 - val_loss: 0.0451
Epoch 328/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0092 - val_loss: 0.0450
Epoch 329/500
52/52 [==============================] - 1s 14ms/step - loss: 0.0092 - val_loss: 0

52/52 [==============================] - 1s 15ms/step - loss: 0.0073 - val_loss: 0.0445
Epoch 481/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0076 - val_loss: 0.0443
Epoch 482/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0080 - val_loss: 0.0445
Epoch 483/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0076 - val_loss: 0.0443
Epoch 484/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0077 - val_loss: 0.0449
Epoch 485/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0076 - val_loss: 0.0435
Epoch 486/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0078 - val_loss: 0.0449
Epoch 487/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0084 - val_loss: 0.0448
Epoch 488/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0075 - val_loss: 0.0439
Epoch 489/500
52/52 [==============================] - 1s 15ms/step - loss: 0.0088 - val_loss: 0